In [1]:
import pandas as pd
import numpy as np
import ast
import csv
import time

from operator import itemgetter
import networkx as nx
from networkx.algorithms import community

In [2]:
users=np.zeros((40001,1))

In [3]:
users.shape

(40001, 1)

In [4]:
user_friends={}

In [5]:
max_friends=0
min_friends=200000

In [6]:
def hasMoreTags(s):
    return s[-1]!=']'

In [7]:
user_tags={}

In [8]:
start=time.time()
with open('data.csv', 'r') as infile:
    reader = csv.reader(infile)
    count=-1
    
    for row in reader:
        #print(row[:7])
        count += 1
        if count == 0:
            continue
        id=row[0]
        #print(count)
        users[count-1][0]=id
        screenName=row[1]
        
        many_tags=hasMoreTags(row[2])
        
        offset=0
        #print('{} originally'.format(row[2]))
        #print(row[3])
        if many_tags:
            this_tags=[]
            for j in range(20):
                this_tags.append(row[2+j].strip('[]').strip(''))
                if row[2+j][-1]==']':
                    break
            #print('{} many tags'.format(this_tags))
            user_tags[id]=this_tags
            offset=len(this_tags)-1
            #print(offset)
        else:
            user_tags[id]=row[2].strip('[]').strip('')
            
        #followersCount=row[4]
        #print(offset)
        friendsCount=int(row[5+offset])
        #print(row[5+offset])
        max_friends=max(max_friends,friendsCount)
        min_friends=min(min_friends,friendsCount)
        friends=row[9+offset:]
        friends[0]=friends[0].strip('[]')
        friends[-1]=friends[-1].strip('[]')
        friends=[i.replace('"', '') for i in friends]
        friends=[i.strip() for i in friends]
        user_friends[id]=friends
        #print("{} : {}".format(count, friends))
end=time.time()
print('{} seconds'.format(end-start))

16.954005479812622 seconds


In [9]:
print(max_friends)
print(min_friends)

238749
101


In [10]:
dict(list(user_friends.items())[0:2])

{'1969527638': ['1969574754',
  '1969295556',
  '1969284056',
  '1969612214',
  '1970067476',
  '1969797386',
  '1969430539',
  '1969840064',
  '1969698176',
  '1970005154',
  '283011644',
  '1969901029',
  '1969563175',
  '1969302314',
  '1969978662',
  '1969457936',
  '1969667533',
  '1969547821',
  '1969943478',
  '1969668032',
  '283006529',
  '1969809440',
  '1969601096',
  '1969298856',
  '1969331652',
  '1969385498',
  '1969674368',
  '1969565263',
  '1970144676',
  '1969745390',
  '1969947438',
  '1969734134',
  '1969801326',
  '1969324008',
  '1969259820',
  '1969535827',
  '1970072989',
  '1969771688',
  '1969437804',
  '1969507394',
  '1969509972',
  '1969751588',
  '283012808',
  '1969302888',
  '1970224440',
  '1969603532',
  '283011244',
  '1969501046',
  '1969887518',
  '1970153138',
  '1970267527',
  '1969941955',
  '1969421654',
  '1970013110',
  '1969544905',
  '1969839590',
  '1969876500',
  '1969674625',
  '1969337952',
  '1970046536',
  '1970090934',
  '1969419133'

In [11]:
dict(list(user_tags.items())[0:110])

{'1969527638': ' "#nationaldogday" ',
 '51878493': ' "#nationaldogday" ',
 '1393409100': ' "#narcos" ',
 '232891415': ' "#gloryoutnow" ',
 '710130422907207680': ' "#nationaldogday" ',
 '3649469655': ' "#veranomtv2016" ',
 '99769502': ' "felipe massa" ',
 '413415713': ' "#narcos" ',
 '518053777': ' "#nationaldogday" ',
 '3167964531': ' "#nationaldogday" ',
 '634516322': ' "#getwellsoonjackson" ',
 '3157134163': ' "#justinssextapeleakedparty" ',
 '1852404296': ' "galaxy note 7" ',
 '320733081': ' "#nationaldogday" ',
 '611525786': ' "#backtohogwarts" ',
 '451819350': ' "#backtohogwarts" ',
 '465857810': ' "#backtohogwarts" ',
 '1969604874': ' "#backtohogwarts" ',
 '1238939516': ' "#getwellsoonjackson" ',
 '177694573': ' "britney" ',
 '4781323279': ' "#narcos" ',
 '299828914': ' "#backtohogwarts" ',
 '2728660403': ' "#nationaldogday" ',
 '353480695': ' "#prettylittleliars" ',
 '1687385503': ' "#nationaldogday" ',
 '2422549237': ' "cape canaveral" ',
 '1282070173': ' "#replacetÃ¸plyricswit

In [12]:
# Generate the nodes and edges
nodes=user_friends.keys()

In [13]:
len(nodes)

40000

In [14]:
node_names=[]

In [15]:
# Generate the node names
with open('data.csv', 'r') as infile:
    reader = csv.reader(infile)
    count=-1
    
    for row in reader:
        count+=1
        if count==0:
            continue
        node_names.append(row[1])

In [16]:
node_names

['LlngoMakeEmCum_',
 '_notmichelle',
 'jesseayye',
 'MrBrianLloyd',
 'sarahdorat_16',
 'wanderIustregui',
 'andhesonit',
 'Jas_Thxku',
 'KLitzau',
 'ThePettyHomo',
 'darkgreyxv',
 'michellemichmic',
 'FreeFX2',
 'RichLantos',
 'lonesomestereo',
 'topnotchcvlt',
 'm_e_lees',
 'reml_conway',
 'HobiHobiHaz',
 'BeyonceStanBish',
 'mrtzchris09',
 'ramzax10',
 'a_sewart',
 'ambreezye',
 'trisetella',
 'aguilarmigs',
 'e_loochie',
 'ViktorSeriogin',
 'oliviaaajames',
 'ghoulslayers',
 'UrosRancic',
 'zunurain214',
 'Meagan_Grieger',
 'tytheintern',
 'WWMobileNews',
 'MCsnipes',
 'Naktadogg',
 'cooperjellybean',
 'taeftjimin',
 'BIEBERnRIRI',
 'fiona_langdon',
 'EmmanCabs',
 'hannahturner19x',
 'KARMAWOWZA',
 'lilacjaime',
 'V4apese',
 'ThePerfectPad57',
 'queermerlxn',
 'KellerDwight',
 'emailthebert',
 'eebeniro',
 'KiusRaie',
 'mjsty_',
 'Teri0328',
 'princesssam93',
 'Holliewrightx',
 'emmkov',
 'celtic_stephen',
 'xJanosklansx_UK',
 'iam_ybnl',
 'randomafstuff',
 'be975n_',
 'mattate9',
 

In [17]:
# Generate the edges
start=time.time()
edges=[]

for u in user_friends:
    for v in user_friends[u]:
        edges.append(tuple((u,v)))
end=time.time()
print('{} seconds'.format(end-start))

8.4595468044281 seconds


In [18]:
edges

[('1969527638', '1969574754'),
 ('1969527638', '1969295556'),
 ('1969527638', '1969284056'),
 ('1969527638', '1969612214'),
 ('1969527638', '1970067476'),
 ('1969527638', '1969797386'),
 ('1969527638', '1969430539'),
 ('1969527638', '1969840064'),
 ('1969527638', '1969698176'),
 ('1969527638', '1970005154'),
 ('1969527638', '283011644'),
 ('1969527638', '1969901029'),
 ('1969527638', '1969563175'),
 ('1969527638', '1969302314'),
 ('1969527638', '1969978662'),
 ('1969527638', '1969457936'),
 ('1969527638', '1969667533'),
 ('1969527638', '1969547821'),
 ('1969527638', '1969943478'),
 ('1969527638', '1969668032'),
 ('1969527638', '283006529'),
 ('1969527638', '1969809440'),
 ('1969527638', '1969601096'),
 ('1969527638', '1969298856'),
 ('1969527638', '1969331652'),
 ('1969527638', '1969385498'),
 ('1969527638', '1969674368'),
 ('1969527638', '1969565263'),
 ('1969527638', '1970144676'),
 ('1969527638', '1969745390'),
 ('1969527638', '1969947438'),
 ('1969527638', '1969734134'),
 ('1969527

In [19]:
len(edges)

32887245

In [23]:
# Build the graph
start=time.time()
G=nx.Graph()
G.add_edges_from(edges)
end=time.time()
print('{} seconds'.format(end-start))

1208.2369549274445 seconds


In [20]:
# Write to pickle
# nx.write_gpickle(G,"test.gpickle")
begin=time.time()
# Read from pickle
G=nx.read_gpickle("test.gpickle")
end=time.time()
print('Spent {} in reading graph'.format(end-begin))

Spent 1085.3727645874023 in reading graph


In [29]:
users=user_friends.keys()

In [43]:
users=list(users)

In [38]:
# Get the minimum distances
count=0
shortest_dis=1
size=len(users)
print(size)
has_no_paths={}
distances=[]

40000


In [ ]:
for u in range(1000):
    for v in range(1000):
        if u == v:
            continue
        count+=1
        # print('{}, {}'.format(u,v))
        node_u=users[u]
        node_v=users[v]
        
        if nx.has_path(G, source=node_u, target=node_v) is False:
            has_no_paths[node_u]=node_v
            continue
        
        cur_dis=nx.shortest_path_length(G,source=node_u,target=node_v)
        distances.append(cur_dis)
        shortest_dis=max(shortest_dis,cur_dis)
        # print(shortest_dis)

In [ ]:
print(nx.info(G))

In [12]:
# Save graph for Cytoscape use
# nx.write_graphml(G, "graph.graphml")

In [ ]:
# Calculate the shortest path
begin=time.time()
length=nx.all_pairs_shortest_path_length(G)
end=time.time()
print('Spend {} seconds in calculating the shortest path'.format(end-begin))
print(length[1][3])

In [ ]:
# Set attributes to nodes
nx.set_node_attributes(G,)

In [26]:
import plotly.graph_objects as go

In [ ]:
Xn=[layt[k][0] for k in range(100)]
Yn=[layt[k][1] for k in range(100)]
Xe=[]
Ye=[]
for e in E:
    Xe+=[layt[e[0]][0],layt[e[1]][0], None]
    Ye+=[layt[e[0]][1],layt[e[1]][1], None]

In [ ]:
trace1=Scatter(x=Xe,
               y=Ye,
               mode='lines',
               line= dict(color='rgb(210,210,210)', width=1),
               hoverinfo='none'
               )

In [165]:
! pip install plotly

  Created wheel for retrying: filename=retrying-1.3.3-cp37-none-any.whl size=11435 sha256=14bef5f985001914ac649fb3c318cbbd685a69fd28b0c7cd88730c461f267d37
  Stored in directory: C:\Users\tingzhang.2019\AppData\Local\pip\Cache\wheels\d7\a9\33\acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying


In [166]:
pos=nx.get_node_attributes(G,'pos')

In [167]:
# Get Node Positions
dmin=1
ncenter=0
for n in pos:
    x,y=pos[n]
    d=(x-0.5)**2+(y-0.5)**2
    if d<dmin:
        ncenter=n
        dmin=d
        
p=nx.single_source_shortest_path_length(G,ncenter)

NodeNotFound: Source 0 is not in G

In [172]:
import pygraphviz as pgv

ModuleNotFoundError: No module named 'pygraphviz'

In [ ]:
! pip install pygraphviz

In [25]:
nx.draw(G)
plt.savefig("simple_path.png") # save as png
plt.show() 

MemoryError: 

In [ ]:
this_graph=pgv.AGraph()
this_graph.add_edges_from(edges)
end=time.time()
temp=end-start
print('{} second'.format(temp))

In [ ]:
print(this_graph.string()) # print to screen
print("Wrote simple.dot")
this_graph.write('simple.dot') # write to simple.dot

B=pgv.AGraph('simple.dot') # create a new graph from file
B.layout() # layout with default (neato)
B.draw('simple.png') # draw png
print("Wrote simple.png")